# AWS AI Week : OpenCALM SageMaker Fine-tuning

SageMaker上でLoRAにより [OpenCALM](https://huggingface.co/cyberagent/open-calm-7b) を Fine-tuning し、デプロイするサンプルコードです。
- このデモでは、Instruction tuning 形式で Fine-tuning を行います。
- [Databricks Dolly 15k](https://github.com/databrickslabs/dolly/tree/master/data) データセットを日本語に翻訳したものを利用します。　(License: [Creative Commons Attribution-ShareAlike 3.0 Unported License](https://creativecommons.org/licenses/by-sa/3.0/legalcode))。
- Dolly dataset は、質問への回答(オープン/クローズ)、分類、要約、情報抽出、ブレインストーミングなど、約 15,000 のさまざまなカテゴリの Instruction レコードを含んでいます。
- ユースケースに対応するために、category が 「closed_qa」(与えた input を元にした質問の回答)、「information_extraction」(与えた input から情報を抽出), 「summarization」(要約) であるデータを抽出したデータセットを学習に使用します。

- 以下のようなユースケースを想定しています。
  - 日本語対応の大規模言語モデルを使いたい
  - 情報はあるがあまり貯めてない
  - 情報の種類はバラバラ (ニュース、フィードバック、議事録、顧客情報など)
  - やることは「情報を抜き出す」「まとめる」
  - 小さく始めたい

SageMaker Studio のノートブックカーネルは、 Data Science 2.0、Python 3 で動作確認済みです。

---

まず、必要なパッケージをインストールしてアップグレードします。以下のセルを初回に実行したら、カーネルを再起動します。

---

In [ ]:
!pip install -U "sagemaker>=2.143.0"

In [ ]:
import sagemaker, boto3, json
from sagemaker import get_execution_role
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.huggingface import HuggingFace

role = get_execution_role()
region = boto3.Session().region_name
sess = sagemaker.Session()
bucket = sess.default_bucket()

sagemaker.__version__

## 学習用データセットの準備
---
Fine-Tuning 用の日本語データをダウンロードします。

---

In [ ]:
!curl -L https://huggingface.co/datasets/kunishou/databricks-dolly-15k-ja/resolve/main/databricks-dolly-15k-ja.json --create-dirs -o ./data/databricks-dolly-15k-ja.json

In [ ]:
!head ./data/databricks-dolly-15k-ja.json

---

データセットをS3へアップロードします。

- 以下を実行したら、直接作成した json ファイルの内容も確認してみてください。
- 元データ (databricks-dolly-15k-ja.json)、および、作成済みデータ (databricks-dolly-15k-ja_extracted.json) は、「data」フォルダに格納されます。

---

In [ ]:
# jsonの要素数を確認
with open('./data/databricks-dolly-15k-ja.json') as f:
    jsondata = json.load(f)
print("もとの要素数：", len(jsondata))

# categoryを指定してデータを抽出する
extracted_data = []
for d in jsondata:
    if d['category'] in ['closed_qa', 'information_extraction', 'summarization']:
        extracted_data.append(d)
        
with open('./data/databricks-dolly-15k-ja_extracted.json', 'w') as f:
    json.dump(extracted_data, f, ensure_ascii=False, indent=2)      

print("抽出後の要素数：", len(extracted＿data))
    
# 最初の5000件のインデックスの要素を保存する場合
# with open('./data/databricks-dolly-15k-ja_first_5000.json', 'w') as f:
#    json.dump(jsondata[:5000], f, ensure_ascii=False)
    
# 最初の5000件を除いたインデックスの要素を保存する場合
# with open('./data/databricks-dolly-15k-ja_after_5000.json', 'w') as f:
#    json.dump(jsondata[5000:], f, ensure_ascii=False)

In [ ]:
# 学習させるデータを指定してS3へアップロード
input_train = sess.upload_data(
    # path="./data/databricks-dolly-15k-ja.json",
    path="./data/databricks-dolly-15k-ja_extracted.json",
    key_prefix="Dolly"
)
input_train

## Fine-tuning を実行
---
作成したデータセットを使用して、Fine-tuning を行います。

今回は、[HuggingFace estimator](https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/sagemaker.huggingface.html#hugging-face-estimator) (推定器) を使用して、学習を行います。

- entry_point : 学習のエントリポイントとして実行する Python ソースファイルへのパス(source＿dir)
- instance_type : 学習で使用するインスタンスタイプを指定

主なハイパーパラメータを説明します。
- base_model : Fine-tuning に使用するモデルを指定
- data_path : Fine-tuning に使用するデータを指定
- num_epochs : 1 epochは、データセット全体が1回だけ渡される(小さなバッチに分割して送信される)
- LoRA (Low Rank Adapter)
  凍結した事前学習済みモデルに低ランク行列を追加し、低ランク行列に対してパラメータ更新する手法。
  - lora_target_modules : チューニング用のパラメーターを用意する
  - lora_r : LoRA の行列ランクであり、小さいほど学習によって更新されるパラメータは少なくなる。
- prompt_template_name : プロンプトのテンプレートを指定(scripts/code/templates/)

---

In [ ]:
base_job_name="OpenCALM"
hyperparameters={
    'base_model':'cyberagent/open-calm-7b', # OpenCALM 7B モデルを使用
    # 'load_in_8bit': True,
    'load_in_4bit': True,
    'pad_token_id': 1,
    'data_path': '/opt/ml/input/data/train/databricks-dolly-15k-ja_extracted.json', # 要素を調整したデータセットを指定
    'num_epochs': 1, # default 3
    'cutoff_len': 512,
    'group_by_length': False,
    'output_dir': '/opt/ml/model',
    # 'resume_from_checkpoint': '/opt/ml/checkpoints',
    'lora_target_modules': '[query_key_value]',
    'lora_r': 16,
    'batch_size': 32,
    'micro_batch_size': 4,
    'prompt_template_name': 'alpaca',
}

In [ ]:
huggingface_estimator = HuggingFace(
    base_job_name=base_job_name,
    role=role,
    entry_point='finetune.py',
    source_dir='./scripts/code',
    # instance_type='ml.g5.2xlarge',
    instance_type='ml.p3.2xlarge', # 小さい GPU インスタンスに変更
    instance_count=1,
    volume_size=200,
    transformers_version='4.26',
    pytorch_version='1.13',
    py_version='py39',
    use_spot_instances=False,
    # use_spot_instances=True, # スポットインスタンスを利用する場合は True にし、max_wait を有効にする。
    # max_wait=86400,
    hyperparameters=hyperparameters,
    metric_definitions=[{'Name': 'eval_loss', 'Regex': "'eval_loss': (\d\.\d+)"},
                        {'Name': 'train_loss', 'Regex': "'loss': (\d\.\d+)"}],
    # checkpoint_s3_uri=f"s3://{bucket}/{base_job_name}/checkpoint/",
)
huggingface_estimator.fit({'train': input_train})

## トレーニングされたモデルを取得
---

In [ ]:
import boto3
import sagemaker

# 最後に実行されたトレーニングジョブで出力されたアーティファクトを取得して S3 に保存
def get_latest_training_job_artifact(base_job_name):
    sagemaker_client = boto3.client('sagemaker')
    response = sagemaker_client.list_training_jobs(NameContains=base_job_name, SortBy='CreationTime', SortOrder='Descending')
    training_job_arn = response['TrainingJobSummaries'][0]['TrainingJobArn']
    training_job_description = sagemaker_client.describe_training_job(TrainingJobName=training_job_arn.split('/')[-1])
    return training_job_description['ModelArtifacts']['S3ModelArtifacts']

try:
    model_data = huggingface_estimator.model_data
except:
    # カーネルがリスタートした時にアーティファクトの　URL を取得
    model_data = get_latest_training_job_artifact('OpenCALM')
    
!aws s3 cp {model_data} opencalm.tar.gz

In [ ]:
!rm -rf scripts/model && mkdir scripts/model
!tar -xvf opencalm.tar.gz -C scripts/model --no-same-owner --wildcards adapter_*
!ls -l scripts/model

## モデルをパッケージして S3 へアップロード
---

In [ ]:
%cd scripts
!tar -czvf ../package.tar.gz *
%cd -

In [ ]:
model_path = sess.upload_data('package.tar.gz', bucket=bucket, key_prefix=f"OpenCALM")
model_path

## パッケージ化された Fine-tuning 済みのモデルをデプロイする
---
リアルタイムエンドポイントを作成し、エンドポイント経由で推論できるようにデプロイします。

[リアルタイム推論](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/realtime-endpoints.html)は、リアルタイム、インタラクティブ、ミリ秒オーダーの低レイテンシーが要求されるワークロードに最適です。ペイロードサイズは最大 6 MB です。

デプロイオプションとして、[非同期推論 (async inference)](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/async-inference.html) を行うこともできます。

非同期推論もエンドポイント経由でモデルへアクセスしますが、リクエストをキューに配置して非同期で処理します。ペイロードサイズがリアルタイム推論よりも大きい(最大 1 GB )、処理時間が長い(最大 1 時間)、ほぼリアルタイムの要求に最適です。処理するリクエストがない場合は、オートスケーリングで最少インスタンス数をゼロにすることができるため、コストを節約する事ができます。

---

In [ ]:
from sagemaker.async_inference import AsyncInferenceConfig
from sagemaker.serializers import JSONSerializer

endpoint_name = "OpenCALM"

huggingface_model = PyTorchModel(
    model_data=model_path,
    framework_version="1.13",
    py_version='py39',
    role=role,
    name=endpoint_name,
    env={
        "model_params": json.dumps({
            "base_model": "cyberagent/open-calm-7b", # モデルを指定
            "lora_weights": "model", # モデルパッケージを取得するためのパス
            "peft": True,
            "load_4bit": True,
            "prompt_template": "alpaca",
        }),
        "SAGEMAKER_MODEL_SERVER_TIMEOUT": "3600"
    }
)

# SageMaker 推論として、モデルをデプロイする
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type='ml.g5.2xlarge',
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
    # 非同期推論の場合は、以下オプションを使用する
    # async_inference_config=AsyncInferenceConfig()
)

## 推論を実行
---
実際に推論を行なってみます。
SageMaker SDK と Boto3 でのサンプルコードを用意していますが、セルをコピーし、様々な入力を試してください。
- max_new_tokens: モデルは、出力長 (入力コンテキストの長さを除く) が max_new_tokens に達するまでテキストを生成する。指定する場合は正の整数でなければならない。
- temperature: 出力のランダム性を制御する。temperature が高いと確率の低い単語での出力になり(より創造的)、temperature が低いと確率の高いワードで出力される。
- top_p: テキスト生成の各ステップで、累積確率が top_p をもつ最小の単語セットからサンプリングする。(0.1の場合は、確率が上位10%を考慮。) 値は、0〜1 の浮動小数点数。
- return_full_text: True の場合、入力テキストが出力テキストの一部として出力される。

---

In [ ]:
# With SageMaker SDK

from sagemaker.predictor import Predictor
from sagemaker.predictor_async import AsyncPredictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor_client = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)
# predictor_client = AsyncPredictor(
#     predictor=predictor_client,
#     name=endpoint_name
# )
data = {
    "instruction": "ヴァージン・オーストラリアはいつから運航を開始したのですか？",
    "input": "ヴァージン・オーストラリア航空（Virgin Australia Airlines Pty Ltd）の商号で、オーストラリアを拠点とする航空会社です。ヴァージン・ブランドを使用する航空会社の中で、保有機材数では最大の航空会社である。2000年8月31日にヴァージン・ブルーとして、2機の航空機で単一路線で運航を開始した[3]。2001年9月のアンセット・オーストラリアの破綻後、突然オーストラリア国内市場の大手航空会社としての地位を確立した。その後、ブリスベン、メルボルン、シドニーをハブとして、オーストラリア国内の32都市に直接乗り入れるまでに成長した[4]。",
    "max_new_tokens": 256,
    "temperature": 0.3,
    "do_sample": True,
    "pad_token_id": 1,
    "bos_token_id": 0,
    "eos_token_is": 0,
    # "repetition_penalty": 1.05,
    # "top_p": 0.75,
    # "top_k": 40,
    # "no_repeat_ngram_size": 2,
    "stop_ids": [1, 0],
}
response = predictor_client.predict(
    data=data
)
print(response)

In [ ]:
# With Boto3

import boto3
import json

endpoint_name = "OpenCALM"
sagemaker_client = boto3.client('sagemaker-runtime')

data = {
    "instruction": "ヴァージン・オーストラリアはいつから運航を開始したのですか？",
    "input": "ヴァージン・オーストラリア航空（Virgin Australia Airlines Pty Ltd）の商号で、オーストラリアを拠点とする航空会社です。ヴァージン・ブランドを使用する航空会社の中で、保有機材数では最大の航空会社である。2000年8月31日にヴァージン・ブルーとして、2機の航空機で単一路線で運航を開始した[3]。2001年9月のアンセット・オーストラリアの破綻後、突然オーストラリア国内市場の大手航空会社としての地位を確立した。その後、ブリスベン、メルボルン、シドニーをハブとして、オーストラリア国内の32都市に直接乗り入れるまでに成長した[4]。",
    "max_new_tokens": 128,
    "temperature": 0.7,
    "do_sample": True,
    "pad_token_id": 1,
    "bos_token_id": 0,
    "eos_token_is": 0,
    # "top_p": 0.9,
    # "repetition_penalty": 1.05,
    "stop_ids": [50278, 50279, 50277, 1, 0],
}

response = sagemaker_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Accept='application/json',
    Body=json.dumps(data)
)

result = json.loads(response['Body'].read())
print(result)

- インプット例 (SageMaker SDK)：

In [ ]:
# サンプルのインプット With SageMaker SDK
#　長文を元に、指定の内容を抽出する、文書を要約する

data = {
    "instruction": "カナダについて簡単に説明してください",
    "input": "カナダは、北米に位置する国である。10の州と3つの準州が大西洋から太平洋、北極海まで広がり、総面積は世界第2位で、世界最長の海岸線を持つ国である。気象学的にも地質学的にも幅広い地域があることが特徴である。人口は少なく、大半は55度線以南の都市部に住んでいます。首都はオタワ、三大都市圏はトロント、モントリオール、バンクーバーです。\n\n現在のカナダには、何千年もの間、先住民族が住み続けています。16世紀に入ると、イギリスとフランスの探検隊が大西洋岸を探検し、後に定住しました。様々な武力衝突の結果、フランスは1763年に北米のほぼすべての植民地を割譲しました。1867年、北アメリカの3つのイギリス植民地が連合し、4つの州からなる連邦制のカナダが誕生しました。1931年に制定されたウェストミンスター憲章に端を発し、1982年に制定されたカナダ法によって、英国議会への法的依存は解消されました。\n\nカナダは、議会制自由民主主義国家であり、ウェストミンスターの伝統を受け継ぐ立憲君主制国家でもあります。首相は、選挙で選ばれた下院の信任を得る能力によって就任し、国家元首であるカナダ君主を代表する総督によって「召集」される。英連邦の領域であり、連邦管区では公式に英語とフランス語の2ヶ国語表記となっています。政府の透明性、生活の質、経済競争力、イノベーション、教育などの国際的な測定において、非常に高い評価を得ている。また、大規模な移民の受け入れにより、世界で最も民族的多様性に富んだ多文化国家のひとつとなっています。米国との長く複雑な関係は、カナダの歴史、経済、文化に大きな影響を及ぼしています。\n\n先進国であるカナダは、一人当たりの名目所得が世界最高水準にあり、豊富な天然資源と整備された国際貿易網を主な財源とする先進国として、世界最大級の経済規模を誇っています。カナダは、国連、NATO、G7、G10、G20、経済協力開発機構（OECD）、世界貿易機関（WTO）、英連邦、北極評議会、国際フランコフォニー機関、アジア太平洋経済協力フォーラム、アメリカ国家機構などの主要国際機関や政府間組織に加盟しています。",
    "max_new_tokens": 256,
    "temperature": 0.3,
    "do_sample": True,
    "pad_token_id": 1,
    "bos_token_id": 0,
    "eos_token_is": 0,
    # "repetition_penalty": 1.05,
    # "top_p": 0.75,
    # "top_k": 40,
    # "no_repeat_ngram_size": 2,
    "stop_ids": [1, 0],
}
response = predictor_client.predict(
    data=data
)
print(response)


## ベンチマーク： Speed

In [ ]:
%timeit response = predictor_client.predict(data=data)

## エンドポイントを削除 (クリーニング)

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()